# XGBoost AFT + 5-Model Ensemble (Independent Seeds & Hyperparameters)

- Model: XGBoost AFT ("survival:aft")
- Ensemble: 5 models, each independently tuned with Optuna (different seeds)
- Metric: IPCW C-index (accounts for censoring)
- Validation: Nested CV
  - Outer CV: evaluation of generalization
  - Inner CV: hyperparameter optimization per model
- Ensembling: rank-based aggregation of risk scores
- Statistical evaluation: mean, std, 95% CI, paired t-test

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sksurv.util import Surv
from sksurv.metrics import concordance_index_ipcw

import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
from scipy import stats

# Reproducibility
np.random.seed(42)

# Global configuration
N_MODELS_ENSEMBLE = 5        # number of models in the ensemble
N_SPLITS_OUTER = 5           # number of outer folds
N_SPLITS_INNER = 3           # number of inner folds for Optuna
N_TRIALS_INNER = 30          # number of Optuna trials per model per outer fold
TAU_CINDEX = 7.0             # time horizon for IPCW C-index

# 1. Data Loading & Preprocessing

In [7]:
# Load enhanced train and validation data
train_enhanced = pd.read_csv("../../data/train_enhanced.csv", sep=",")
eval_enhanced = pd.read_csv("../../data/eval_enhanced.csv", sep=",")

print(f"Train data shape: {train_enhanced.shape}")
print(f"Validation data shape: {eval_enhanced.shape}")
print("\nTrain columns:")
print(train_enhanced.columns.tolist())

Train data shape: (3173, 237)
Validation data shape: (1193, 235)

Train columns:
['ANC', 'ANC_missing', 'BM_BLAST', 'BM_BLAST_missing', 'CHR_12_count', 'CHR_17_count', 'CHR_1_count', 'CHR_20_count', 'CHR_21_count', 'CHR_2_count', 'CHR_4_count', 'CHR_5_count', 'CHR_7_count', 'CHR_X_count', 'CHR_nunique', 'DELLEN_sum', 'DEPTH_avg', 'DEPTH_max', 'DEPTH_min', 'DEPTH_std', 'EFFECT_FV_count', 'EFFECT_LOF_ratio', 'EFFECT_NS_count', 'EFFECT_SG_count', 'EFFECT_nunique', 'HB', 'HB_missing', 'ID', 'LEN_avg', 'LEN_max', 'MONOCYTES', 'MONOCYTES_missing', 'Nmut', 'PLT', 'PLT_missing', 'VAF_avg', 'VAF_max', 'VAF_std', 'WBC', 'WBC_missing', 'baseline_chr_count', 'eln_like_flag_adverse_cyto', 'eln_like_flag_intermediate_cyto', 'eln_like_risk_cyto', 'has_abn17p', 'has_any_abnormality', 'has_inv16_or_t_16_16', 'has_inv3_or_t3_3', 'has_minus5_or_del5q', 'has_minus7_or_del7q', 'has_plus8', 'has_t_15_17', 'has_t_6_9', 'has_t_8_21', 'has_t_9_22', 'is_abnormal_karyotype', 'is_complex_karyotype', 'is_cyto_miss

In [8]:
time_col = "OS_YEARS"
event_col = "OS_STATUS"

exclude_cols = {time_col, event_col, "ID"}
feature_cols = [c for c in train_enhanced.columns if c not in exclude_cols]

# Features: train
X_df = train_enhanced[feature_cols].astype(float)
X_df = X_df.replace([np.inf, -np.inf], np.nan).fillna(0.0)
X = X_df.to_numpy(dtype=float)

# Features: validation
X_eval_df = eval_enhanced[feature_cols].astype(float)
X_eval_df = X_eval_df.replace([np.inf, -np.inf], np.nan).fillna(0.0)
X_eval = X_eval_df.to_numpy(dtype=float)

# Targets (raw time & event)
time_vals = train_enhanced[time_col].to_numpy(dtype=float)
event_vals = train_enhanced[event_col].to_numpy(dtype=bool)  # True = event, False = censored

# Log-time for AFT
y_log_time = np.log1p(time_vals)

print(f"\nX shape: {X.shape}")
print(f"X_eval shape: {X_eval.shape}")
print(f"Number of events: {event_vals.sum()} / {len(event_vals)}")


X shape: (3173, 234)
X_eval shape: (1193, 234)
Number of events: 1600 / 3173


# 2. Utilities for XGBoost AFT & Risk Scoring

In [9]:
def set_global_seeds(seed: int):
    """Set global NumPy seed (XGBoost uses 'seed' param internally)."""
    np.random.seed(seed)


def make_aft_bounds(log_times: np.ndarray, events: np.ndarray):
    """
    Build lower and upper bounds for XGBoost AFT.
    - For uncensored (event=1): lb = ub = log_time
    - For right-censored (event=0): lb = log_time, ub = +inf
    """
    lb = log_times.copy()
    ub = log_times.copy()
    ub[~events] = np.inf  # event == False => censored => upper bound +inf
    return lb, ub


def make_dmatrix(X_data: np.ndarray, log_times: np.ndarray, events: np.ndarray):
    """
    Create an XGBoost DMatrix with AFT lower and upper bounds.
    """
    lb, ub = make_aft_bounds(log_times, events)
    dmat = xgb.DMatrix(X_data)
    dmat.set_float_info("label_lower_bound", lb)
    dmat.set_float_info("label_upper_bound", ub)
    return dmat


def xgb_aft_risk_from_log_pred(log_pred: np.ndarray) -> np.ndarray:
    """
    Convert predicted log-time to a risk score:
    - Higher predicted survival time => lower risk.
    - We define risk = -log_pred.
    """
    return -log_pred


def compute_ipcw_cindex_from_risk(
    time_train: np.ndarray,
    event_train: np.ndarray,
    time_test: np.ndarray,
    event_test: np.ndarray,
    risk_scores: np.ndarray,
    tau: float = TAU_CINDEX,
):
    """
    Compute IPCW C-index using sksurv.metrics.concordance_index_ipcw.
    """
    y_train_struct = Surv.from_arrays(event=event_train.astype(bool), time=time_train)
    y_test_struct = Surv.from_arrays(event=event_test.astype(bool), time=time_test)
    cindex = concordance_index_ipcw(
        y_train_struct,
        y_test_struct,
        risk_scores,
        tau=tau,
    )[0]
    return cindex


def rank_based_ensemble(risk_matrix: np.ndarray) -> np.ndarray:
    """
    Rank-based ensemble:
      - risk_matrix: shape (n_models, n_samples)
      - For each model, convert risk scores to ranks (1 = lowest risk, N = highest risk),
        then average ranks across models.
    """
    n_models, n_samples = risk_matrix.shape
    ranks_all = np.zeros_like(risk_matrix, dtype=float)

    for m in range(n_models):
        scores = risk_matrix[m]
        order = np.argsort(scores)  # ascending (lowest risk -> rank 1)
        ranks = np.empty_like(order, dtype=float)
        ranks[order] = np.arange(1, n_samples + 1)
        ranks_all[m] = ranks

    mean_ranks = ranks_all.mean(axis=0)
    return mean_ranks

# 3. Nested CV: Outer (Evaluation) / Inner (Optuna) + 5-Model Ensemble

In [10]:
outer_kf = KFold(n_splits=N_SPLITS_OUTER, shuffle=True, random_state=123)

cv_single_scores = []    # single model = first model of ensemble
cv_ensemble_scores = []  # ensemble of 5 models (rank-based)

print("=== Starting outer cross-validation for XGBoost AFT ensemble ===")

for fold, (train_idx, test_idx) in enumerate(outer_kf.split(X), 1):
    print(f"\n######## Outer Fold {fold}/{N_SPLITS_OUTER} ########")

    # Outer train/test splits
    X_train_outer = X[train_idx]
    X_test_outer = X[test_idx]

    time_train_outer = time_vals[train_idx]
    time_test_outer = time_vals[test_idx]

    event_train_outer = event_vals[train_idx]
    event_test_outer = event_vals[test_idx]

    y_log_train_outer = y_log_time[train_idx]
    y_log_test_outer = y_log_time[test_idx]

    # Ensemble storage for this outer fold
    risk_models_test = []
    single_cindex_this_fold = None  # C-index of the first model

    for m in range(N_MODELS_ENSEMBLE):
        print(f"\n  >> Ensemble model {m+1}/{N_MODELS_ENSEMBLE} (outer fold {fold})")

        base_seed = 1000 + 100 * fold + m
        set_global_seeds(base_seed)

        inner_kf = KFold(
            n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
        )

        def inner_objective(trial):
            """
            Optuna objective for inner CV.
            For each trial, we evaluate average IPCW C-index over inner folds.
            """
            params = {
                "objective": "survival:aft",
                "eval_metric": "aft-nloglik",
                "learning_rate": trial.suggest_float(
                    "learning_rate", 0.01, 0.3, log=True
                ),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float(
                    "colsample_bytree", 0.5, 1.0
                ),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "gamma": trial.suggest_float("gamma", 0.0, 5.0),
                "aft_loss_distribution": trial.suggest_categorical(
                    "aft_loss_distribution", ["normal", "logistic", "extreme"]
                ),
                "aft_loss_distribution_scale": trial.suggest_float(
                    "aft_loss_distribution_scale", 0.5, 2.0
                ),
                "tree_method": "hist",
                "seed": base_seed,
            }

            num_boost_round = trial.suggest_int("num_boost_round", 50, 500)

            inner_c_indices = []

            for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
                inner_kf.split(X_train_outer), 1
            ):
                # Inner train/val split
                X_train_inner = X_train_outer[inner_train_idx]
                X_val_inner = X_train_outer[inner_val_idx]

                time_train_inner = time_train_outer[inner_train_idx]
                time_val_inner = time_train_outer[inner_val_idx]

                event_train_inner = event_train_outer[inner_train_idx]
                event_val_inner = event_train_outer[inner_val_idx]

                y_log_train_inner = y_log_train_outer[inner_train_idx]
                y_log_val_inner = y_log_train_outer[inner_val_idx]

                # Build DMatrices with AFT bounds
                dtrain_inner = make_dmatrix(
                    X_train_inner, y_log_train_inner, event_train_inner
                )
                dval_inner = make_dmatrix(
                    X_val_inner, y_log_val_inner, event_val_inner
                )

                # Train model for this inner fold
                try:
                    model_inner = xgb.train(
                        params,
                        dtrain_inner,
                        num_boost_round=num_boost_round,
                        evals=[(dval_inner, "valid")],
                        verbose_eval=False,
                    )
                except Exception:
                    inner_c_indices.append(0.5)
                    continue

                # Predictions and risk scores
                y_pred_log_val = model_inner.predict(dval_inner)
                risk_val = xgb_aft_risk_from_log_pred(y_pred_log_val)

                if np.isnan(risk_val).any():
                    inner_c_indices.append(0.5)
                    continue

                # IPCW C-index
                try:
                    cindex_inner = compute_ipcw_cindex_from_risk(
                        time_train_inner,
                        event_train_inner,
                        time_val_inner,
                        event_val_inner,
                        risk_val,
                        tau=TAU_CINDEX,
                    )
                except Exception:
                    cindex_inner = 0.5

                inner_c_indices.append(cindex_inner)

            return float(np.mean(inner_c_indices))

        # Optuna study for this model on this outer fold
        study = optuna.create_study(
            direction="maximize",
            study_name=f"xgb_aft_outer_fold{fold}_model{m}",
        )
        study.optimize(
            inner_objective,
            n_trials=N_TRIALS_INNER,
            show_progress_bar=False,
        )

        best_params = study.best_params
        print(f"    - Best inner CV C-index : {study.best_value:.4f}")
        print(f"    - Best hyperparameters  : {best_params}")

        # Extract num_boost_round and form final param dict
        best_num_boost_round = best_params.pop("num_boost_round")
        final_params = best_params.copy()
        final_params.update(
            {
                "objective": "survival:aft",
                "eval_metric": "aft-nloglik",
                "tree_method": "hist",
                "seed": base_seed,
            }
        )

        # Final training on full outer-train set
        dtrain_outer = make_dmatrix(
            X_train_outer, y_log_train_outer, event_train_outer
        )

        try:
            model_m = xgb.train(
                final_params,
                dtrain_outer,
                num_boost_round=best_num_boost_round,
                verbose_eval=False,
            )
        except Exception:
            print("    ⚠ Training failed on outer train set. Model ignored.")
            continue

        # Predict on outer-test
        dtest_outer = xgb.DMatrix(X_test_outer)
        y_pred_log_test = model_m.predict(dtest_outer)
        risk_test = xgb_aft_risk_from_log_pred(y_pred_log_test)
        risk_models_test.append(risk_test)

        # Define "single model" as the first ensemble member
        if m == 0:
            if np.isnan(risk_test).any():
                cindex_single = 0.5
            else:
                try:
                    cindex_single = compute_ipcw_cindex_from_risk(
                        time_train_outer,
                        event_train_outer,
                        time_test_outer,
                        event_test_outer,
                        risk_test,
                        tau=TAU_CINDEX,
                    )
                except Exception:
                    cindex_single = 0.5
            single_cindex_this_fold = cindex_single
            print(f"    -> Single model IPCW C-index on outer test : {cindex_single:.4f}")

    # End for each ensemble model

    # Ensemble C-index for this outer fold
    if len(risk_models_test) == 0:
        print("  ⚠ No valid models for this outer fold. Ensemble C-index = 0.5.")
        cindex_ensemble = 0.5
    else:
        risk_matrix_test = np.vstack(risk_models_test)
        risk_ensemble = rank_based_ensemble(risk_matrix_test)

        if np.isnan(risk_ensemble).any():
            cindex_ensemble = 0.5
        else:
            try:
                cindex_ensemble = compute_ipcw_cindex_from_risk(
                    time_train_outer,
                    event_train_outer,
                    time_test_outer,
                    event_test_outer,
                    risk_ensemble,
                    tau=TAU_CINDEX,
                )
            except Exception:
                cindex_ensemble = 0.5

    if single_cindex_this_fold is None:
        single_cindex_this_fold = 0.5

    cv_single_scores.append(single_cindex_this_fold)
    cv_ensemble_scores.append(cindex_ensemble)

    print(f"\n  Outer fold {fold} summary:")
    print(f"    Single model IPCW C-index : {single_cindex_this_fold:.4f}")
    print(f"    Ensemble IPCW C-index     : {cindex_ensemble:.4f}")

[I 2025-12-30 03:52:28,713] A new study created in memory with name: xgb_aft_outer_fold1_model0


=== Starting outer cross-validation for XGBoost AFT ensemble ===

######## Outer Fold 1/5 ########

  >> Ensemble model 1/5 (outer fold 1)


[I 2025-12-30 03:52:29,868] Trial 0 finished with value: 0.7121196189148744 and parameters: {'learning_rate': 0.08412131790493813, 'max_depth': 8, 'subsample': 0.9743670129127417, 'colsample_bytree': 0.9031449373363049, 'min_child_weight': 1, 'gamma': 1.8878083509544146, 'aft_loss_distribution': 'extreme', 'aft_loss_distribution_scale': 1.1513905135550164, 'num_boost_round': 122}. Best is trial 0 with value: 0.7121196189148744.
[I 2025-12-30 03:52:31,297] Trial 1 finished with value: 0.7216310859227573 and parameters: {'learning_rate': 0.011647156676752446, 'max_depth': 10, 'subsample': 0.5145437219350621, 'colsample_bytree': 0.5403687337676062, 'min_child_weight': 2, 'gamma': 4.785712655026881, 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': 1.7197037564758213, 'num_boost_round': 425}. Best is trial 1 with value: 0.7216310859227573.
[I 2025-12-30 03:52:36,078] Trial 2 finished with value: 0.7098464907745908 and parameters: {'learning_rate': 0.01702010918016363, 'max_

    - Best inner CV C-index : 0.7223
    - Best hyperparameters  : {'learning_rate': 0.019759364169074665, 'max_depth': 3, 'subsample': 0.5958737243083626, 'colsample_bytree': 0.5001293996610701, 'min_child_weight': 5, 'gamma': 3.5155768476969147, 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': 1.6444159926901754, 'num_boost_round': 220}


[I 2025-12-30 03:53:12,439] A new study created in memory with name: xgb_aft_outer_fold1_model1


...

    - Best inner CV C-index : 0.7235
    - Best hyperparameters  : {'learning_rate': 0.02159161115090783, 'max_depth': 9, 'subsample': 0.827912870308325, 'colsample_bytree': 0.7066147955468285, 'min_child_weight': 8, 'gamma': 2.4868650514420914, 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': 1.485286832145341, 'num_boost_round': 398}


[I 2025-12-30 04:10:41,666] A new study created in memory with name: xgb_aft_outer_fold5_model4



  >> Ensemble model 5/5 (outer fold 5)


[I 2025-12-30 04:10:43,134] Trial 0 finished with value: 0.7142436284458555 and parameters: {'learning_rate': 0.025812376919674723, 'max_depth': 4, 'subsample': 0.7801588604640618, 'colsample_bytree': 0.5955013008464447, 'min_child_weight': 10, 'gamma': 0.39419997097332127, 'aft_loss_distribution': 'extreme', 'aft_loss_distribution_scale': 1.2807262324315911, 'num_boost_round': 314}. Best is trial 0 with value: 0.7142436284458555.
[I 2025-12-30 04:10:44,827] Trial 1 finished with value: 0.6983249738296808 and parameters: {'learning_rate': 0.07679939329715986, 'max_depth': 9, 'subsample': 0.7306477088603938, 'colsample_bytree': 0.7569977684338522, 'min_child_weight': 8, 'gamma': 0.32312282984537144, 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': 0.9198903688226567, 'num_boost_round': 155}. Best is trial 0 with value: 0.7142436284458555.
[I 2025-12-30 04:10:46,546] Trial 2 finished with value: 0.7210115577379793 and parameters: {'learning_rate': 0.010090273584633063, '

    - Best inner CV C-index : 0.7229
    - Best hyperparameters  : {'learning_rate': 0.025803262182652303, 'max_depth': 4, 'subsample': 0.7069238782879057, 'colsample_bytree': 0.6343296534398603, 'min_child_weight': 9, 'gamma': 3.8604535920051632, 'aft_loss_distribution': 'normal', 'aft_loss_distribution_scale': 1.1102821947225556, 'num_boost_round': 353}

  Outer fold 5 summary:
    Single model IPCW C-index : 0.7087
    Ensemble IPCW C-index     : 0.7097


# 4. Statistical Analysis: Means, 95% CI, Paired t-test

In [22]:
def summarize_scores(scores, name: str):
    scores = np.array(scores)
    mean = scores.mean()
    std = scores.std(ddof=1)
    n = len(scores)

    t_crit = stats.t.ppf(0.975, df=n - 1)
    ci_low = mean - t_crit * std / np.sqrt(n)
    ci_high = mean + t_crit * std / np.sqrt(n)

    print(f"\n{name}")
    print(f"  Scores by fold : {np.round(scores, 4)}")
    print(f"  Mean           : {mean:.4f}")
    print(f"  Std dev        : {std:.4f}")
    print(f"  95% CI         : [{ci_low:.4f} ; {ci_high:.4f}]")
    return scores, mean, std, (ci_low, ci_high)


single_scores, single_mean, single_std, single_ci = summarize_scores(
    cv_single_scores, "Single model (first model of ensemble)"
)
ensemble_scores, ensemble_mean, ensemble_std, ensemble_ci = summarize_scores(
    cv_ensemble_scores, "Ensemble (5 models, rank-based)"
)

# Paired t-test: ensemble vs single
diff = ensemble_scores - single_scores
t_stat, p_val = stats.ttest_rel(ensemble_scores, single_scores)

print("\nPaired t-test: Ensemble vs Single model")
print(f"  Fold-wise differences (ensemble - single) : {np.round(diff, 4)}")
print(f"  Mean difference : {diff.mean():.4f}")
print(f"  t-statistic     : {t_stat:.4f}")
print(f"  p-value         : {p_val:.4f}")
if p_val < 0.05:
    print("  => Statistically significant improvement at 5% level")
else:
    print("  => NOT statistically significant at 5% level with these folds")


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Visualization with Plotly
folds = np.arange(1, N_SPLITS_OUTER + 1)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("C-index per outer fold", "Average performance (±1 SD)"),
    horizontal_spacing=0.15  # tu peux garder ici ton spacing custom si tu veux
)

# ---- Left subplot: fold-wise scores ----
fig.add_trace(
    go.Scatter(
        x=folds,
        y=single_scores,
        mode="lines+markers",
        name="Single model",
        marker=dict(symbol="circle", size=10),
        line=dict(width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=folds,
        y=ensemble_scores,
        mode="lines+markers",
        name="Ensemble (rank)",
        marker=dict(symbol="square", size=10),
        line=dict(width=2)
    ),
    row=1, col=1
)

fig.update_xaxes(title_text="Outer fold", row=1, col=1)
fig.update_yaxes(title_text="IPCW C-index", row=1, col=1)

# ---- Right subplot: bar plot with std ----
fig.add_trace(
    go.Bar(
        x=["Single", "Ensemble"],
        y=[single_mean, ensemble_mean],
        error_y=dict(
            type="data",
            array=[single_std, ensemble_std],
            visible=True
        ),
        name="Mean ± 1 SD"
    ),
    row=1, col=2
)

fig.update_yaxes(title_text="IPCW C-index (mean)", row=1, col=2)

fig.update_layout(
    width=900,
    height=400,
    showlegend=True,
    title_text="Model comparison: single vs ensemble",
    template="plotly_white",
    margin=dict(l=60, r=60, t=80, b=50)
)

fig.show()



Single model (first model of ensemble)
  Scores by fold : [0.7258 0.7426 0.7245 0.7096 0.7087]
  Mean           : 0.7223
  Std dev        : 0.0139
  95% CI         : [0.7050 ; 0.7395]

Ensemble (5 models, rank-based)
  Scores by fold : [0.7279 0.7424 0.7283 0.7134 0.7097]
  Mean           : 0.7243
  Std dev        : 0.0131
  95% CI         : [0.7080 ; 0.7406]

Paired t-test: Ensemble vs Single model
  Fold-wise differences (ensemble - single) : [ 0.0021 -0.0002  0.0037  0.0038  0.001 ]
  Mean difference : 0.0021
  t-statistic     : 2.7004
  p-value         : 0.0541
  => NOT statistically significant at 5% level with these folds


# 5. Final Training on Full Train Set + 5-Model Ensemble for Validation Predictions

In [15]:
def optimize_and_train_full_xgb_aft(
    X_data: np.ndarray,
    time_data: np.ndarray,
    event_data: np.ndarray,
    log_time_data: np.ndarray,
    base_seed: int,
):
    """
    On the full dataset:
      - Inner CV to optimize hyperparameters (with Optuna)
      - Final training on the full data with best hyperparameters
    Returns the trained model, best params, and loss proxy (training eval metric traced by XGBoost).
    """
    set_global_seeds(base_seed)

    inner_kf = KFold(
        n_splits=N_SPLITS_INNER, shuffle=True, random_state=base_seed
    )

    def inner_objective_full(trial):
        params = {
            "objective": "survival:aft",
            "eval_metric": "aft-nloglik",
            "learning_rate": trial.suggest_float(
                "learning_rate", 0.01, 0.3, log=True
            ),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float(
                "colsample_bytree", 0.5, 1.0
            ),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "aft_loss_distribution": trial.suggest_categorical(
                "aft_loss_distribution", ["normal", "logistic", "extreme"]
            ),
            "aft_loss_distribution_scale": trial.suggest_float(
                "aft_loss_distribution_scale", 0.5, 2.0
            ),
            "tree_method": "hist",
            "seed": base_seed,
        }

        num_boost_round = trial.suggest_int("num_boost_round", 50, 500)

        inner_c_indices = []

        for inner_fold, (inner_train_idx, inner_val_idx) in enumerate(
            inner_kf.split(X_data), 1
        ):
            X_train_inner = X_data[inner_train_idx]
            X_val_inner = X_data[inner_val_idx]

            time_train_inner = time_data[inner_train_idx]
            time_val_inner = time_data[inner_val_idx]

            event_train_inner = event_data[inner_train_idx]
            event_val_inner = event_data[inner_val_idx]

            y_log_train_inner = log_time_data[inner_train_idx]
            y_log_val_inner = log_time_data[inner_val_idx]

            dtrain_inner = make_dmatrix(
                X_train_inner, y_log_train_inner, event_train_inner
            )
            dval_inner = make_dmatrix(
                X_val_inner, y_log_val_inner, event_val_inner
            )

            try:
                model_inner = xgb.train(
                    params,
                    dtrain_inner,
                    num_boost_round=num_boost_round,
                    evals=[(dval_inner, "valid")],
                    verbose_eval=False,
                )
            except Exception:
                inner_c_indices.append(0.5)
                continue

            y_pred_log_val = model_inner.predict(dval_inner)
            risk_val = xgb_aft_risk_from_log_pred(y_pred_log_val)

            if np.isnan(risk_val).any():
                inner_c_indices.append(0.5)
                continue

            try:
                cindex_inner = compute_ipcw_cindex_from_risk(
                    time_train_inner,
                    event_train_inner,
                    time_val_inner,
                    event_val_inner,
                    risk_val,
                    tau=TAU_CINDEX,
                )
            except Exception:
                cindex_inner = 0.5

            inner_c_indices.append(cindex_inner)

        return float(np.mean(inner_c_indices))

    # Optuna on full dataset (inner CV)
    study_full = optuna.create_study(
        direction="maximize", study_name=f"xgb_aft_full_seed{base_seed}"
    )
    study_full.optimize(
        inner_objective_full,
        n_trials=N_TRIALS_INNER,
        show_progress_bar=False,
    )

    best_params_full = study_full.best_params
    print(f"Seed {base_seed} - best inner CV IPCW C-index: {study_full.best_value:.4f}")
    print(f"Best params: {best_params_full}")

    best_num_boost_round = best_params_full.pop("num_boost_round")
    final_params_full = best_params_full.copy()
    final_params_full.update(
        {
            "objective": "survival:aft",
            "eval_metric": "aft-nloglik",
            "tree_method": "hist",
            "seed": base_seed,
        }
    )

    # Final training on all data
    dtrain_full = make_dmatrix(X_data, log_time_data, event_data)

    model_full = xgb.train(
        final_params_full,
        dtrain_full,
        num_boost_round=best_num_boost_round,
        verbose_eval=True,
    )

    return model_full, final_params_full, best_num_boost_round

In [ ]:
# Train 5 final XGB-AFT models on the full training data
# Using fixed good hyperparameters (no re-optimization for speed)

# Default good hyperparameters for XGB-AFT
DEFAULT_PARAMS = {
    "objective": "survival:aft",
    "eval_metric": "aft-nloglik",
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 3,
    "gamma": 1.0,
    "aft_loss_distribution": "normal",
    "aft_loss_distribution_scale": 1.0,
    "tree_method": "hist",
}
DEFAULT_NUM_BOOST_ROUND = 200

models_final = []
risk_matrix_eval = []

print("\n=== Final XGB-AFT ensemble training (5 models) on the full train set ===")
print("Using fixed hyperparameters (no re-optimization)")

for m in range(N_MODELS_ENSEMBLE):
    base_seed = 5000 + m
    print(f"\n##### Final model {m+1}/{N_MODELS_ENSEMBLE} (seed={base_seed}) #####")
    
    set_global_seeds(base_seed)
    
    # Update seed in params
    params_m = DEFAULT_PARAMS.copy()
    params_m["seed"] = base_seed
    
    # Create DMatrix for full training data
    dtrain_full = make_dmatrix(X, y_log_time, event_vals)
    
    # Train model
    model_m = xgb.train(
        params_m,
        dtrain_full,
        num_boost_round=DEFAULT_NUM_BOOST_ROUND,
        verbose_eval=50,
    )
    
    models_final.append(model_m)
    
    # Predictions on validation set
    dval_final = xgb.DMatrix(X_eval)
    y_pred_log_eval = model_m.predict(dval_final)
    risk_eval_m = xgb_aft_risk_from_log_pred(y_pred_log_eval)
    risk_matrix_eval.append(risk_eval_m)
    
    print(f"  Model {m+1} trained successfully.")

print(f"\n=== All {N_MODELS_ENSEMBLE} models trained! ===")

[I 2025-12-30 04:25:40,190] A new study created in memory with name: xgb_aft_full_seed5000



=== Final XGB-AFT ensemble training (5 models) on the full train set ===

##### Final model 1/5 (seed=5000) #####


[I 2025-12-30 04:25:42,754] Trial 0 finished with value: 0.7131729159824234 and parameters: {'learning_rate': 0.0265567501639268, 'max_depth': 6, 'subsample': 0.9724637862627878, 'colsample_bytree': 0.7110627835384249, 'min_child_weight': 3, 'gamma': 0.7379054802821478, 'aft_loss_distribution': 'extreme', 'aft_loss_distribution_scale': 1.7347957277485229, 'num_boost_round': 196}. Best is trial 0 with value: 0.7131729159824234.
[W 2025-12-30 04:25:43,014] Trial 1 failed with parameters: {'learning_rate': 0.0703660339762659, 'max_depth': 9, 'subsample': 0.5469788132154916, 'colsample_bytree': 0.7042563723363979, 'min_child_weight': 2, 'gamma': 1.5052949657623578, 'aft_loss_distribution': 'logistic', 'aft_loss_distribution_scale': 0.8010624425281493, 'num_boost_round': 240} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\arthr\Desktop\ENSAE\QRT-Challenge-2025\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_tria

KeyboardInterrupt: 

In [ ]:
risk_matrix_eval = np.array(risk_matrix_eval)
print(f"\nNumber of valid final models in ensemble: {risk_matrix_eval.shape[0]}")

if risk_matrix_eval.shape[0] == 0:
    raise RuntimeError("No valid final model for the XGB-AFT ensemble.")

# Rank-based ensemble on validation set
risk_ensemble_eval = rank_based_ensemble(risk_matrix_eval)

# Optional normalization to [0, 1] for interpretability
risk_min = risk_ensemble_eval.min()
risk_max = risk_ensemble_eval.max()
risk_ensemble_eval_norm = (risk_ensemble_eval - risk_min) / (risk_max - risk_min + 1e-8)

print(f"Min risk (raw ensemble) : {risk_min:.4f}")
print(f"Max risk (raw ensemble) : {risk_max:.4f}")

In [ ]:
# Build final submission / prediction file
submission_tuned = pd.DataFrame({
    "ID": eval_enhanced["ID"],
    "risk_score": risk_ensemble_eval_norm,  # or risk_ensemble_eval if you want raw scores
})

print(submission_tuned.head())

submission_tuned.to_csv("../submissions/submission_xgb_aft_ensemble_nestedcv.csv", index=False)
print("\nPredictions saved to '../submissions/submission_xgb_aft_ensemble_nestedcv.csv'")